<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [30]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [31]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "2ZoLYydQfIz5VW5b.D5B2X2LLCmJsysb4n78mTmGgEAWzlLfrgRKcCLJXlE4iJZ9Mqhm1GhsG60PJQxls"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158296
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "sweden", "street": "sollentuna-sollentunav\\u00e4gen 192", "aqicn_url": "https://api.waqi.info/feed/@13984", "latitude": 17.950061509561397, "longitude": 59.433619224293736}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [32]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
import requests
import pandas as pd
# Read the air quality feature group into a pandas dataframe
air_quality_df = air_quality_fg.read()
print(air_quality_df.iloc[-1])

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
# Convert the date column to include timezone information
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'], utc=True)
print(aq_today_df)

# Merge the lag columns into the aq_today_df
aq_today_df = aq_today_df.merge(air_quality_df[['date', 'pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3']], on='date', how='left')

aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.90s) 
                       date  pm25  pm25_lag_1  pm25_lag_2  pm25_lag_3 country  \
0 2023-09-26 00:00:00+00:00  22.0        14.0         8.0        13.0  sweden   
1 2024-10-09 00:00:00+00:00  16.0        25.0        16.0        11.0  sweden   
2 2024-08-20 00:00:00+00:00  19.0        17.0        17.0        11.0  sweden   
3 2024-08-10 00:00:00+00:00  12.0        15.0        17.0        22.0  sweden   
4 2024-04-27 00:00:00+00:00  24.0        10.0        21.0        34.0  sweden   

     city                          street                                url  
0  sweden  sollentuna-sollentunavägen 192  https://api.waqi.info/feed/@13984  
1  sweden  sollentuna-sollentunavägen 192  https://api.waqi.info/feed/@13984  
2  sweden  sollentuna-sollentunavägen 192  https://api.waqi.info/feed/@13984  
3  sweden  sollentuna-sollentunavägen 192  https://api.waqi.info/feed/@13984  
4  sweden  sollentuna-sollentuna

,pm25,country,city,street,date,url,pm25_lag_1,pm25_lag_2,pm25_lag_3
0,16.0,sweden,sweden,sollentuna-sollentunavägen 192,2024-11-13 00:00:00+00:00,https://api.waqi.info/feed/@13984,NaN,NaN,NaN


In [34]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [35]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 18.0°N 59.5°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-13,27.350000,0.0,3.075841,20.556128,sweden
1,2024-11-14,27.350000,0.0,16.575644,34.380402,sweden
2,2024-11-15,28.350000,0.0,21.166653,41.552525,sweden
3,2024-11-16,27.600000,0.0,21.470612,34.755981,sweden
4,2024-11-17,27.400000,0.0,15.277749,34.439060,sweden
5,2024-11-18,28.150000,0.0,26.242987,63.083534,sweden
6,2024-11-19,29.000000,0.0,22.383780,62.198643,sweden
7,2024-11-20,27.950001,0.1,24.696104,59.322803,sweden
8,2024-11-21,27.500000,0.0,19.228851,51.842735,sweden
9,2024-11-22,26.850000,0.0,17.057314,44.145004,sweden


In [36]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pm25_lag_1 (type: 'float') is missing from input dataframe.
 - pm25_lag_2 (type: 'float') is missing from input dataframe.
 - pm25_lag_3 (type: 'float') is missing from input dataframe.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.

In [ ]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-12 13:48:36,459 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1158296/fs/1148999/fg/1345937


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1158296/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x2be14dcd0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669735
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T12:48:36.000459Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
